## Exploratory Data Analysis

In [1]:
#### Import Relevant Packages
import pandas as pd
import numpy as np

#!pip install category_encoders
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

#!pip install xgboost
from xgboost import XGBClassifier

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
## Load Data Set
df = pd.read_csv(r"C:\Users\basil.p.sony\OneDrive - Accenture\Self\Sigmoid\Data\train_(6).csv")
label = pd.read_csv(r"C:\Users\basil.p.sony\OneDrive - Accenture\Self\Sigmoid\Data\train_churn_labels.csv")

In [3]:
### Checking for Attributes with greater than 50% Null Values and removing unwanted columns
threshold = 0.15
nan_percentage = df.isna().mean()
columns_with_low_nan = nan_percentage[nan_percentage < threshold].index.tolist()
print(f"Columns with less than 15% NaN: {columns_with_low_nan}")

columns_below_threshold = (nan_percentage < threshold).sum()
print(f"Number of columns with less than 15% NaN: {columns_below_threshold}")
df1=df[columns_with_low_nan]

Columns with less than 15% NaN: ['Var6', 'Var7', 'Var13', 'Var21', 'Var22', 'Var24', 'Var25', 'Var28', 'Var35', 'Var38', 'Var44', 'Var57', 'Var65', 'Var73', 'Var74', 'Var76', 'Var78', 'Var81', 'Var83', 'Var85', 'Var109', 'Var112', 'Var113', 'Var119', 'Var123', 'Var125', 'Var132', 'Var133', 'Var134', 'Var140', 'Var143', 'Var144', 'Var149', 'Var153', 'Var160', 'Var163', 'Var173', 'Var181', 'Var192', 'Var193', 'Var195', 'Var196', 'Var197', 'Var198', 'Var199', 'Var202', 'Var203', 'Var204', 'Var205', 'Var206', 'Var207', 'Var208', 'Var210', 'Var211', 'Var212', 'Var216', 'Var217', 'Var218', 'Var219', 'Var220', 'Var221', 'Var222', 'Var223', 'Var226', 'Var227', 'Var228']
Number of columns with less than 15% NaN: 66


In [4]:
#### Separating Numerical and Categorical Values
categorical_columns = df1.select_dtypes(include=['object', 'category'])
numerical_columns = df1.select_dtypes(include=['number'])
print(len(categorical_columns.columns))
print(len(numerical_columns.columns))

28
38


In [39]:
## Descriptive Stats for NUmerical Column
numerical_columns.describe().to_csv('summary_stats.csv')

In [5]:
##### Checking for outliers in numerical columns
def count_outliers(dataframe):
    outlier_counts = {}
    for column in dataframe:  
        Q1 = dataframe[column].quantile(0.25)
        Q3 = dataframe[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 2.5 * IQR
        upper_bound = Q3 + 2.5 * IQR
        outliers = dataframe[(dataframe[column] < lower_bound) | (dataframe[column] > upper_bound)][column]
        outlier_counts[column] = outliers.count()
    return outlier_counts

outlier_counts = count_outliers(numerical_columns)
print(outlier_counts)


{'Var6': 2102, 'Var7': 967, 'Var13': 1978, 'Var21': 3112, 'Var22': 2973, 'Var24': 1287, 'Var25': 1739, 'Var28': 488, 'Var35': 3772, 'Var38': 22, 'Var44': 670, 'Var57': 0, 'Var65': 1625, 'Var73': 0, 'Var74': 4164, 'Var76': 402, 'Var78': 3929, 'Var81': 208, 'Var83': 820, 'Var85': 1621, 'Var109': 3036, 'Var112': 2208, 'Var113': 5895, 'Var119': 2852, 'Var123': 1009, 'Var125': 2062, 'Var132': 8252, 'Var133': 106, 'Var134': 1244, 'Var140': 3545, 'Var143': 396, 'Var144': 21, 'Var149': 1226, 'Var153': 0, 'Var160': 1977, 'Var163': 1731, 'Var173': 134, 'Var181': 3001}


In [6]:
####Replace all outliers and Nans by Median of the column

def replace_outliers_with_median(dataframe):
    for column in dataframe:
        Q1 = dataframe[column].quantile(0.25)
        Q3 = dataframe[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 2.5 * IQR
        upper_bound = Q3 + 2.5 * IQR
        median = dataframe[column].median()

        # Replace outliers with the median
        dataframe[column] = np.where(
            (dataframe[column] < lower_bound) | (dataframe[column] > upper_bound), ##| (dataframe[column].isna()),
            median,
            dataframe[column]
        )
    return dataframe

df_cleaned = replace_outliers_with_median(numerical_columns)
df_cleaned = df_cleaned.fillna(0)

print(df_cleaned.head())

     Var6  Var7   Var13  Var21  Var22  Var24  Var25   Var28  Var35      Var38  \
0  1526.0   7.0   184.0  464.0  580.0   14.0  128.0  166.56    0.0     3570.0   
1   525.0   0.0     0.0  168.0  210.0    2.0   24.0  353.52    0.0  4764966.0   
2   861.0   7.0   904.0  144.0  180.0    2.0   48.0  220.08    0.0  5883894.0   
3     0.0   0.0     0.0    0.0    0.0    0.0    0.0   22.08    0.0        0.0   
4  1029.0   7.0  3216.0   64.0   80.0    4.0   64.0  200.00    0.0        0.0   

   Var44     Var57  Var65  Var73  Var74      Var76  Var78      Var81  Var83  \
0    0.0  4.076907    9.0   36.0   35.0  1350864.0    0.0    7333.11    5.0   
1    0.0  5.408032    9.0   26.0    0.0  2872928.0    0.0  151098.90   25.0   
2    0.0  6.599658    9.0  130.0    7.0  1675776.0    0.0   16211.58   40.0   
3    0.0  1.988250    9.0   12.0    0.0        0.0    0.0       0.00    0.0   
4    0.0  4.552446   18.0   82.0  224.0   784448.0    0.0   37423.50    0.0   

   Var85  Var109  Var112     Var113  V

In [7]:
zero_columns = df_cleaned.columns[df_cleaned.sum() == 0]
df_cleaned_v1 = df_cleaned.drop(columns = zero_columns)
df_cleaned_v1.shape

(50000, 31)

## Feature Engineering

In [8]:
### Identify categorical columns with low and high cardinality
columns_with_few_unique_values = categorical_columns.columns[categorical_columns.nunique() < 5]
low_cardinality = categorical_columns[columns_with_few_unique_values]
high_cardinality = categorical_columns.drop(columns = columns_with_few_unique_values)

In [9]:
### Binary Encoding for high cardinality attributes
encoder = ce.BinaryEncoder(cols=high_cardinality.columns)
df_encoded_high_card = encoder.fit_transform(high_cardinality)
df_encoded_high_card.shape

(50000, 168)

In [10]:
### One Hot Encoding for Low cardinality attributes
df_encoded_low_card = pd.get_dummies(low_cardinality, columns=low_cardinality.columns,drop_first = True)
df_encoded_low_card.shape

(50000, 11)

In [11]:
### Check for multi colinearity
corelation_mat = df_cleaned_v1.corr()
#corelation_mat.to_csv("corr_mat.csv")
#print(corelation_mat)

high_corelation = corelation_mat[abs(corelation_mat) > 0.9]
#print(high_corelation)

threshold = 0.9
highly_correlated_cols = []

for column in corelation_mat.columns:
    correlated_features = corelation_mat.index[corelation_mat[column] > threshold].tolist()
    correlated_features = [feature for feature in correlated_features if feature != column]
    highly_correlated_cols.extend(correlated_features)
    
highly_correlated_cols = list(set(highly_correlated_cols))
print(highly_correlated_cols)

['Var22', 'Var21']


In [12]:
df_final_model = df_cleaned_v1.drop(columns = highly_correlated_cols)
df_final_model.shape

(50000, 29)

In [13]:
### Combine all numerical and encoded data sets
df_encoded_final = pd.concat([df_final_model,df_encoded_low_card], axis = 1)
df_encoded_final.shape

(50000, 40)

## Feature Selection

In [17]:
### Use Random Forest Variable Importance for Feature Selection

X=df_encoded_final
y=label["Label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True)

class_weights = {-1: 1, 1: 5}
rf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight = 'balanced')
rf.fit(X, y)

RandomForestClassifier(class_weight='balanced', n_estimators=200,
                       random_state=42)

In [18]:
### Check Feature Importance
importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

              Feature  Importance
7               Var57    0.095416
17             Var113    0.066446
9               Var73    0.057798
12              Var81    0.049533
5               Var28    0.042515
0                Var6    0.042213
26             Var153    0.041180
18             Var119    0.039092
21             Var133    0.037915
22             Var134    0.036870
11              Var76    0.034522
20             Var125    0.034501
6               Var38    0.034324
2               Var13    0.031340
23             Var140    0.029419
27             Var160    0.029192
28             Var163    0.028556
4               Var25    0.025291
25             Var149    0.024648
19             Var123    0.023694
16             Var112    0.021888
10              Var74    0.020904
15             Var109    0.018345
13              Var83    0.017903
14              Var85    0.017178
24             Var144    0.013613
36        Var218_cJvF    0.012441
3               Var24    0.012281
8             

In [20]:
updated_feature = feature_importance_df[feature_importance_df["Importance"]>0.01]["Feature"].tolist()
df_final_model_updated = df_encoded_final[updated_feature]
df_final_model_updated.shape

(50000, 31)

## Model Creation & Evaluation - Random Forest

In [24]:
### Randon Forest Model Training along with Hyper parameter Tuning using Grid Search
X=df_final_model_updated
y=label["Label"]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True)

rf1 = RandomForestClassifier(n_estimators=100, random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt'],
    'class_weight': ['balanced',{0: 1, 1: 5}]
}
grid_search = GridSearchCV(estimator=rf1, param_grid=param_grid, cv=3, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


C:\Users\basil.p.sony\Anaconda3.1\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
108 fits failed out of a total of 216.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
108 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\basil.p.sony\Anaconda3.1\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\basil.p.sony\Anaconda3.1\lib\site-packages\sklearn\ensemble\_forest.py", line 367, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "C:\Users\basil.p.sony\Anaconda3.1\lib\site-packages\sklearn\ensemble\_forest.py", line 783, in _validate_y_cla

Best Parameters: {'class_weight': 'balanced', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Score: 0.9248571762048012


In [25]:
### Use best model parameters to predict
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

#### Classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9225333333333333
Classification Report:
              precision    recall  f1-score   support

          -1       0.93      0.99      0.96     13894
           1       0.19      0.02      0.03      1106

    accuracy                           0.92     15000
   macro avg       0.56      0.51      0.49     15000
weighted avg       0.87      0.92      0.89     15000

[[13821    73]
 [ 1089    17]]


In [26]:
### Predicting to check Test and Train Recall to see model performance
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate recall on both training and test data
train_recall = recall_score(y_train, y_train_pred)
test_recall = recall_score(y_test, y_test_pred)

print("Training Recall:", train_recall)
print("Test Recall:", test_recall)

Training Recall: 0.9886983632112237
Test Recall: 0.015370705244122965


## Model Creation & Evaluation - XG Boost

In [27]:
### XG Boost Model Training along with Hyper parameter Tuning using Grid Search
X=df_final_model_updated
y=label["Label"]
y = y.replace(-1,0)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='error', random_state=42)
param_grid = {
    'learning_rate': [0.01, 0.1], 
    'max_depth': [3, 5], 
    'n_estimators': [50, 100, 200],
    'subsample': [0.8, 1.0], 
    'colsample_bytree': [0.8, 1.0],
    'scale_pos_weight': [1,5,10]
}
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, verbose=2, scoring= 'recall')
grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


C:\Users\basil.p.sony\Anaconda3.1\lib\site-packages\xgboost\core.py:158: UserWarning: [18:50:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\basil.p.sony\Anaconda3.1\lib\site-packages\xgboost\core.py:158: UserWarning: [18:50:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, scale_pos_weight=1, subsample=0.8; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, scale_pos_weight=1, subsample=0.8; total time=   0.0s


C:\Users\basil.p.sony\Anaconda3.1\lib\site-packages\xgboost\core.py:158: UserWarning: [18:50:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\basil.p.sony\Anaconda3.1\lib\site-packages\xgboost\core.py:158: UserWarning: [18:50:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, scale_pos_weight=1, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, scale_pos_weight=1, subsample=1.0; total time=   0.0s


...

[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=5, n_estimators=200, scale_pos_weight=10, subsample=1.0; total time=   1.0s


C:\Users\basil.p.sony\Anaconda3.1\lib\site-packages\xgboost\core.py:158: UserWarning: [18:54:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=5, n_estimators=200, scale_pos_weight=10, subsample=1.0; total time=   1.2s


C:\Users\basil.p.sony\Anaconda3.1\lib\site-packages\xgboost\core.py:158: UserWarning: [18:54:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'scale_pos_weight': 10, 'subsample': 0.8}
Best Score: 0.3939876300304239


In [28]:
### Use best model parameters to predict
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

#### Classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.7604666666666666
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.79      0.86     13894
           1       0.14      0.43      0.21      1106

    accuracy                           0.76     15000
   macro avg       0.54      0.61      0.53     15000
weighted avg       0.89      0.76      0.81     15000

[[10935  2959]
 [  634   472]]


In [29]:
### Predicting to check Test and Train Recall to see model performance
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate recall on both training and test data
train_recall = recall_score(y_train, y_train_pred)
test_recall = recall_score(y_test, y_test_pred)

print("Training Recall:", train_recall)
print("Test Recall:", test_recall)

Training Recall: 0.48947778643803586
Test Recall: 0.4267631103074141
